Для любых видео восстановить траекторию движения (t вектор). Выполнить визуализацию. 
Определить параметры которые влияют на "точность" определения вектора t
Использовать решение на базе нейронных сетей. 
Любые идеи. 

Есть видео полета дрона, анализируем с помощью детектора ORB.
Выводим траекторию на трехмерный график.
Результаты сохраняем в таблице csv


In [36]:
# Загрузка необходимых библиотек и двух изображений
import cv2
import numpy as np
import matplotlib.pyplot as plt

def filter_matches_distance(matches, dist_threshold):
    """
    Фильтрация совпавших особенностей из двух изображений по расстоянию между лучшими совпадениями

    Аргументы:
    match -- список совпавших особенностей из двух изображений
    dist_threshold -- максимальное допустимое относительное расстояние между лучшими совпадениями, (0.0, 1.0)

    Возвращает:
    filtered_match -- список хороших совпадений, удовлетворяющих порогу расстояния
    """
    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold * n.distance:
            filtered_match.append(m)

    return filtered_match

def match_features(des1, des2, matching='BF', detector='sift', sort=True, k=2):
    """
    Сопоставление особенностей из двух изображений

    Аргументы:
    des1 -- список дескрипторов ключевых точек на первом изображении
    des2 -- список дескрипторов ключевых точек на втором изображении
    matching -- (str) может быть 'BF' для Brute Force или 'FLANN'
    detector -- (str) может быть 'sift' или 'orb'. По умолчанию 'sift'
    sort -- (bool) сортировать ли совпадения по расстоянию. По умолчанию True
    k -- (int) количество соседей для сопоставления каждой особенности

    Возвращает:
    matches -- список совпадений из двух изображений. Каждое match[i] содержит k или менее совпадений для 
               того же дескриптора запроса

    """
    if matching == 'BF':
        if detector == 'sift':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck=False) # Изменено на NORM_HAMMING
        matches = matcher.knnMatch(des1, des2, k=k)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(des1, des2, k=k)
    
    if sort:
        matches = sorted(matches, key = lambda x:x[0].distance)

    return matches

video_path = 'drone.mp4'  # имя файла (и путь, если необходимо)
myvid = cv2.VideoCapture(video_path) # загрузка в cv2
# Проверка успешного открытия видео
if not myvid.isOpened():
    print("Видео файл не найден")
else:
    print("Видео открыто") 

# матрица камеры K
K = np.array([[3000,   0.    , 960],
              [  0.    , 3000 , 540 ],
              [  0.    ,   0.    ,   1.]], dtype=np.float32)

sift = cv2.SIFT_create()
poses = [] # список для поз камеры
# Список для позиций камеры
positions = [np.array([0, 0, 0])]

# Чтение первого кадра видео
ret, frame = myvid.read()
img1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
i = 0
interval = 20 # интервал между рассматриваемыми кадрами, например, interval = 5 для каждого 5-го кадра
while True: # i < 1000:
    i += 1
    ret, frame = myvid.read() # чтение следующего кадра          
    if ret and i % interval == 0: # каждый 10-й кадр
        print(f'Текущий кадр: {i}')
        img2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        try:
            keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
            keypoints2, descriptors2 = sift.detectAndCompute(img2, None)
            if descriptors1 is None or descriptors2 is None:
                print(f'Нет дескрипторов на кадре {i}')
                continue
            matches = match_features(descriptors1, descriptors2, matching='BF', detector='sift', sort=True)
            matches = filter_matches_distance(matches, 0.7)
            pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
       
            F, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.LMEDS, prob=0.999, threshold=1.0) # LMEDS    
            # F, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0) #RANSAC    
            a, R, t, b = cv2.recoverPose(F, pts1, pts2, K, mask)
            poses.append((R, t))

            # Обновить позицию камеры
            current_position = positions[-1]
            new_position = current_position + np.dot(R, t).T[0]
            positions.append(new_position)
            # Сохранение текущего кадра как предыдущего
            img1 = img2 
        except Exception as e:
            print(f"Ошибка: {e}")
            pass
    elif ret and i % interval != 0:
        pass
    else:
        print('Видео - всё')
        myvid.release()          # освобождение видеоканала после завершения просмотра
        # cv2.destroyAllWindows()  # закрытие всех окон
        break                    # выход из цикла

print(poses)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def create_trajectory(poses):
    trajectory = [np.array([0, 0, 0])]
    current_pose = np.eye(4)
    pose_cam = [np.array([[0, 0, 0],[0, 0, 0],[0, 0, 0]])]

    for R, t in poses:
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = t.T
        current_pose = np.dot(current_pose, T)
        trajectory.append(current_pose[:3, 3])
        pose_cam.append(current_pose[:3, :3])

    return np.array(trajectory), np.array(pose_cam)

trajectory, pose_cam = create_trajectory(poses)

import plotly.graph_objects as go
import numpy as np
# Создание фигуры
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=trajectory[1:, 0],
    y=trajectory[1:, 1],
    z=trajectory[1:, 2],
    mode='lines+markers',
    marker=dict(size=3, color='blue'),
    line=dict(color='blue', width=2),
    name='Camera Trajectory'
))

# Добавление ориентации камеры в каждой точке траектории
for i, (R, t) in enumerate(zip(pose_cam, trajectory)):
    # Направление камеры (ось Z камеры)
    camera_direction = R @ np.array([0, 0, 1])  # Направление оси Z камеры
    camera_direction_end = t + camera_direction * 0.5  # Конец вектора направления

    # Добавляем линию, представляющую направление камеры
    fig.add_trace(go.Scatter3d(
        x=[t[0], camera_direction_end[0]],
        y=[t[1], camera_direction_end[1]],
        z=[t[2], camera_direction_end[2]],
        mode='lines',
        line=dict(color='green', width=2),
        name=f'Camera Direction {i}' if i == 0 else None,
        showlegend=False if i > 0 else True
    ))

# Настройка макета графика
fig.update_layout(
    title='Camera Motion Trajectory',
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    showlegend=True
)

# Показать график
fig.show()


Видео открыто
Текущий кадр: 20
Текущий кадр: 40
Текущий кадр: 60
Текущий кадр: 80
Текущий кадр: 100
Текущий кадр: 120
Текущий кадр: 140
Текущий кадр: 160
Текущий кадр: 180
Текущий кадр: 200
Текущий кадр: 220
Текущий кадр: 240
Текущий кадр: 260
Текущий кадр: 280
Текущий кадр: 300
Текущий кадр: 320
Текущий кадр: 340
Текущий кадр: 360
Текущий кадр: 380
Текущий кадр: 400
Текущий кадр: 420
Текущий кадр: 440
Текущий кадр: 460
Текущий кадр: 480
Текущий кадр: 500
Текущий кадр: 520
Текущий кадр: 540
Текущий кадр: 560
Текущий кадр: 580
Текущий кадр: 600
Текущий кадр: 620
Текущий кадр: 640
Текущий кадр: 660
Текущий кадр: 680
Текущий кадр: 700
Текущий кадр: 720
Текущий кадр: 740
Текущий кадр: 760
Текущий кадр: 780
Текущий кадр: 800
Текущий кадр: 820
Текущий кадр: 840
Текущий кадр: 860
Текущий кадр: 880
Текущий кадр: 900
Текущий кадр: 920
Текущий кадр: 940
Текущий кадр: 960
Текущий кадр: 980
Текущий кадр: 1000
Текущий кадр: 1020
Текущий кадр: 1040
Текущий кадр: 1060
Текущий кадр: 1080
Текущий кадр:

Мы использовали детектор ORB для анализа видео и расчета 3D-траектории движения дрона и ориентации его камеры. Для нахождения эссенциальной матрицы применялись методы cv2.LMEDS и cv2.RANSAC. Точность расчета зависит от интервала между обрабатываемыми кадрами и метода расчета эссенциальной матрицы. Например, при анализе каждого пятого кадра траектория выглядела достаточно плавной, и результаты также были хорошими

---